In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from ReweightingHelper import ReweightingHelper
from AltSetupHandler import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x92fdac0


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
# prod = False
prod = True
no_rvec = True
# write_outputs = False
write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/selected-objects/test.json"
output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/test"
output_meta_path = "data/datasets/reweighted"
output_meta = f"{output_meta_path}/test.json"
checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = [
#     "true_lep_lvec", "true_nu_lvec", "true_quark1_lvec", "true_quark2_lvec",
#     "iso_lep_lvec", "nu_lvec", "R2Jet_sel1_lvec", "R2Jet_sel2_lvec",
#     ]
# true lvecs do not exist in every df so cannot be explicitly requested...
# urgh but empty snapshots are also not allowed
output_collections = r"(true_\w+_lvec)|(iso_lep_lvec)|(nu_lvec)|(R2Jet_sel1_lvec)|(R2Jet_sel2_lvec)|(\w*sqme\w*)|(weight\w*)"
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/selected-objects/signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/signal-only"
    output_meta_path = "data/datasets/reweighted"
    output_meta = f"{output_meta_path}/signal-only.json"
    # checked_output_meta = f"{output_meta_path}/checked-signal-only.json"
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
# ROOT.EnableImplicitMT(n_threads)
environ["OMP_NUM_THREADS"] = "6"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = ReweightingHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xd3b1ec0


In [7]:
# alt_setup_handler = AltSetupHandler("""
# {
#   "SM": {
#     "g1z": 1.0,
#     "ka": 1.0,
#     "la": 0.0
#   },
# "variations": [
#     0.001,
#     0.0001,
#     -0.0001,
#     1e-08
#   ]
# }
# """, mirror=False, combinations=False)
alt_setup_handler = AltSetupHandler("""
{
  "SM": {
    "g1z": 1.0,
    "ka": 1.0,
    "la": 0.0
  },
"variations": [
    0.001,
    -0.001,
    1e-04,
    -1e-04,
    1e-08
  ]
}
""", mirror=False, combinations=False)
alt_configs = alt_setup_handler.get_alt_setup()
print(alt_configs)
analysis.initialise_omega_wrappers(alt_configs)

{'g1z_pos_1em03': {'g1z': 1.001, 'ka': 1.0, 'la': 0.0}, 'ka_pos_1em03': {'g1z': 1.0, 'ka': 1.001, 'la': 0.0}, 'la_pos_1em03': {'g1z': 1.0, 'ka': 1.0, 'la': 0.001}, 'g1z_neg_1em03': {'g1z': 0.999, 'ka': 1.0, 'la': 0.0}, 'ka_neg_1em03': {'g1z': 1.0, 'ka': 0.999, 'la': 0.0}, 'la_neg_1em03': {'g1z': 1.0, 'ka': 1.0, 'la': -0.001}, 'g1z_pos_1em04': {'g1z': 1.0001, 'ka': 1.0, 'la': 0.0}, 'ka_pos_1em04': {'g1z': 1.0, 'ka': 1.0001, 'la': 0.0}, 'la_pos_1em04': {'g1z': 1.0, 'ka': 1.0, 'la': 0.0001}, 'g1z_neg_1em04': {'g1z': 0.9999, 'ka': 1.0, 'la': 0.0}, 'ka_neg_1em04': {'g1z': 1.0, 'ka': 0.9999, 'la': 0.0}, 'la_neg_1em04': {'g1z': 1.0, 'ka': 1.0, 'la': -0.0001}, 'g1z_pos_1em08': {'g1z': 1.00000001, 'ka': 1.0, 'la': 0.0}, 'ka_pos_1em08': {'g1z': 1.0, 'ka': 1.00000001, 'la': 0.0}, 'la_pos_1em08': {'g1z': 1.0, 'ka': 1.0, 'la': 1e-08}}


In [8]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [9]:
min_setups = ["nominal", "g1z_pos_1em08", "ka_pos_1em08", "la_pos_1em08"]

In [10]:
analysis.book_sqme(
                      [
                          "true_beam_e_lvec",
                          "true_beam_p_lvec",
                          "true_lep_lvec",
                          "true_nu_lvec",
                          "true_quark1_lvec",
                          "true_quark2_lvec"
                      ],
                      "true_lep_charge",
                      "mc",
                      categories=signal_category
                     )
# also book sqme for wrong jet assignment
analysis.book_sqme(
                      [
                          "true_beam_e_lvec",
                          "true_beam_p_lvec",
                          "true_lep_lvec",
                          "true_nu_lvec",
                          "true_quark2_lvec",
                          "true_quark1_lvec",
                      ],
                      "true_lep_charge",
                      "wj_mc",
                      alt_setups=min_setups,
                      categories=signal_category
                     )
analysis.book_weights(signal_category)

In [11]:
# define nominal beam lvecs
analysis.Define("nominal_beam_e_lvec", "ROOT::Math::PxPyPzMVector(+8.750143e-01, 0., +1.250000e+02, +5.109968e-04)")
analysis.Define("nominal_beam_p_lvec", "ROOT::Math::PxPyPzMVector(+8.750143e-01, 0., -1.250000e+02, +5.109968e-04)")

In [ ]:
analysis.book_sqme(
                      [
                          "nominal_beam_e_lvec",
                          "nominal_beam_p_lvec",
                          "true_lep_lvec",
                          # FIXME: needs to change to fulfil E,P conservation
                          "true_nu_lvec",
                          "true_quark1_lvec",
                          "true_quark2_lvec"
                      ],
                      "true_lep_charge",
                      "nomb_mc",
                      alt_setups=min_setups,
                      categories=signal_category
                     )
analysis.book_sqme(
                      [
                          "nominal_beam_e_lvec",
                          "nominal_beam_p_lvec",
                          "iso_lep_lvec",
                          # FIXME: needs to change to fulfil E,P conservation
                          "true_nu_lvec",
                          "true_quark1_lvec",
                          "true_quark2_lvec"
                      ],
                      "iso_lep_charge",
                      "nomb_mc_rlep",
                      alt_setups=min_setups,
                      categories=signal_category
                     )

In [13]:
analysis.book_sqme(
                      [
                          "nominal_beam_e_lvec",
                          "nominal_beam_p_lvec",
                          "iso_lep_lvec",
                          "nu_lvec",
                          "R2Jet1_lvec",
                          "R2Jet2_lvec",
                      ],
                      "iso_lep_charge",
                      "reco_12",
                      alt_setups=min_setups,
                     )
analysis.book_sqme(
                      [
                          "nominal_beam_e_lvec",
                          "nominal_beam_p_lvec",
                          "iso_lep_lvec",
                          "nu_lvec",
                          "R2Jet2_lvec",
                          "R2Jet1_lvec",
                      ],
                      "iso_lep_charge",
                      "reco_21",
                      alt_setups=min_setups,
                     )
analysis.book_sqme(
                      [
                          "nominal_beam_e_lvec",
                          "nominal_beam_p_lvec",
                          "iso_lep_lvec",
                          "nu_lvec",
                          "R2Jet_sel1_lvec",
                          "R2Jet_sel2_lvec",
                      ],
                      "iso_lep_charge",
                      "reco_jm",
                      alt_setups=min_setups,
                      categories=signal_category
                     )

In [14]:
analysis.define_optimal_observables_truth([
    "g1z_pos_1em08",
    # "g1z_pos_1em06",
    # "g1z_pos_1em05",
    # "g1z_pos_1em04",
    "ka_pos_1em08",
    "la_pos_1em08",
    # "g1z_pos_1em03",
    # "ka_pos_1em03",
    # "la_pos_1em03",
    ], truth_categories=signal_category)

In [15]:
# analysis.book_histogram_1D("O_g1z_pos_1em08", "O_g1z_pos_1em08", ("", "", 125, -2.5, 2.5))
analysis.book_histogram_1D("mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_g1z_pos_1em06", "mc_O_g1z_pos_1em06", ("", "", 125, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_g1z_pos_1em05", "mc_O_g1z_pos_1em05", ("", "", 125, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_g1z_pos_1em04", "mc_O_g1z_pos_1em04", ("", "", 125, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_ka_pos_1em08", "mc_O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_la_pos_1em08", "mc_O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_g1z_pos_1em03", "mc_O_g1z_pos_1em03", ("", "", 250, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_ka_pos_1em03", "mc_O_ka_pos_1em03", ("", "", 250, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_la_pos_1em03", "mc_O_la_pos_1em03", ("", "", 250, -2.5, 2.5), categories=signal_category)

In [16]:
if write_outputs:
    analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [17]:
%%time
analysis.run()

CPU times: user 8h 47min 28s, sys: 9min 8s, total: 8h 56min 36s
Wall time: 1h 30min 31s


In [18]:
# if write_outputs:
    # analysis.check_snapshots("events", output_path, checked_output_meta)

In [19]:
# analysis.draw_histogram("O_g1z_pos_1em03")
analysis.draw_histogram("mc_O_g1z_pos_1em08", categories=signal_category)
analysis.draw_histogram("mc_O_ka_pos_1em08", categories=signal_category)
analysis.draw_histogram("mc_O_la_pos_1em08", categories=signal_category)

(<cppyy.gbl.THStack object at 0x16b5c210>,
 <cppyy.gbl.TCanvas object at 0x15039080>)

In [20]:
# analysis.compare_histograms_unscaled(["mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em03"], categories=signal_category)
# analysis.compare_histograms_unscaled(["mc_O_ka_pos_1em08", "mc_O_ka_pos_1em03"], categories=signal_category)
# analysis.compare_histograms_unscaled(["mc_O_la_pos_1em08", "mc_O_la_pos_1em03"], categories=signal_category)